<a href="https://colab.research.google.com/github/Ccode104/Artificial-Intelligence/blob/main/Iterative_Deepening_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8-Tile Puzzel problem
Credits : Abhishek Chandurkar (BT22CSE104)

## Helper functions

In [ ]:
import random

In [ ]:
# Goal state
GOAL = (1, 2, 3,
        4, 5, 6,
        7, 8, 0)  # 0 = blank

# Moves: (dx, dy, move name)
# Here , the x axis is pointing down and y axis is pointing right with origin at first element

MOVES = {
    "Up":    (-1, 0),
    "Down":  (1, 0),
    "Left":  (0, -1),
    "Right": (0, 1)
}

In [ ]:
# The tile problem can be checked for existence of a solution by parity check of number of inversions
# If even number of parameters -> solution exists
# else , no solution

def is_solvable(puzzle):
    """Check solvability using inversion count parity."""
    arr = [x for x in puzzle if x != 0]
    inversions = 0
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            if arr[i] > arr[j]:
                inversions += 1
    return inversions % 2 == 0

In [ ]:
def generate_random_puzzle():
    """Generate a random solvable puzzle state."""
    while True:
        puzzle = list(range(9))
        random.shuffle(puzzle)

        # Check if solvable
        if is_solvable(puzzle):
            return tuple(puzzle)
        # if not solvable, take another puzzle

In [ ]:
def get_blank_pos(state):
    """Return (row, col) of blank tile."""
    idx = state.index(0)
    return divmod(idx, 3) # same as (idx//3,idx%3)

In [ ]:
def get_neighbors(state):
    """Generate neighbors of current state with move names."""
    neighbors = []

    # Get the position of the blank tile
    row, col = get_blank_pos(state)

    # We expand the current state to get the children states (or swap the tile with each neighbour)
    for move, (dr, dc) in MOVES.items():
        # New position of the blank tile after the swap
        new_r, new_c = row + dr, col + dc

        # Check if the move is valid (in bounds)
        if 0 <= new_r < 3 and 0 <= new_c < 3:

            # Calculate the index of the blank tile new position (in child state)
            new_idx = new_r * 3 + new_c

            # Calculate the index of blank tile psoition before swap (in parent state)
            blank_idx = row * 3 + col

            # Convert the parent state into list (to make it mutable)
            new_state = list(state)

            # Swap blank with target tile to get the new state(or child)
            new_state[blank_idx], new_state[new_idx] = new_state[new_idx], new_state[blank_idx]

            # We have got a child state
            # Store it in the open list temporarily(it might be a node that was already visited, so may not be considered in open list)
            neighbors.append((tuple(new_state), move))

    return neighbors

## Iterative Deepening Search

In [ ]:
def depth_limited_search(state, limit, path, visited, expanded):
    """Recursive depth-limited search."""
    if state == GOAL:
        return path, expanded

    if limit == 0:
        return None, expanded

    # Get the possible child states
    possible_child_states = get_neighbors(state)

    # Expand the current state
    for neighbor, move in possible_child_states:
        # Check if it was previously visited
        if neighbor not in visited:
            # Mark the child node as visited
            visited.add(neighbor)

            # Keep track of the number of expanded nodes(or states)
            expanded[0] += 1

            # Expand the child node (we reduce the limit by 1, also keep track of the "path" and visited nodes)
            result, expanded = depth_limited_search(neighbor, limit - 1, path + [move], visited, expanded)

            # If we got the goal state, no need to do any more search
            if result is not None:
                return result, expanded
            # else, we remove the nodes that we marked as visited (visualize it as moving backwards)
            visited.remove(neighbor)

    # We could not reach the goal state with given "limit"
    return None, expanded


In [ ]:
def iterative_deepening_search(initial_state):
    """IDS main loop."""
    depth = 0
    expanded = [0]  # keep as list so it's mutable and is passed by reference

    # We try for each "limit" till we get the goal state
    while True:
        visited = {initial_state} # the root node (the given problem state)
        result, expanded = depth_limited_search(initial_state, depth, [], visited, expanded)

        # We found the goal state!
        if result is not None:
            return result, expanded[0]

        # Could not find the goal state for given limit on the depth, so try limit + 1
        depth += 1
    # Here, we do not have a return statement since we know that the given problem is solvable for sure
    # So the return statement in the loop will be executed for sure!

## Bi-directional BFS Approach

In [ ]:
from collections import deque

def bidirectional_search(start):
    if start == GOAL:
        return [], 0

    # Two queues (open lists)
    q_start = deque([start])
    q_goal = deque([GOAL])

    # Parent maps (Or the BFS trees)
    parent_start = {start: (None, None)}  # state -> (parent, move)
    parent_goal = {GOAL: (None, None)}

    expanded = 0

    while q_start and q_goal:
        # Expand from start side
        current_start = q_start.popleft()
        expanded += 1
        for neighbor, move in get_neighbors(current_start):
            # If not visited
            if neighbor not in parent_start:
                # Add the neighbour
                parent_start[neighbor] = (current_start, move)
                q_start.append(neighbor)
                # If the BFS trees meet
                if neighbor in parent_goal:
                    return reconstruct_path(neighbor, parent_start, parent_goal), expanded

        # Expand from goal side (same as the start side)
        current_goal = q_goal.popleft()
        expanded += 1
        for neighbor, move in get_neighbors(current_goal):
            if neighbor not in parent_goal:
                parent_goal[neighbor] = (current_goal, move)
                q_goal.append(neighbor)
                if neighbor in parent_start:
                    return reconstruct_path(neighbor, parent_start, parent_goal), expanded

    return None, expanded


def reconstruct_path(meeting_state, parent_start, parent_goal):
    # Path from start -> meeting
    path_start = []
    state = meeting_state

    # We move backwards from the meeting state to the start state
    while parent_start[state][0] is not None:
        parent, move = parent_start[state]
        path_start.append(move)
        state = parent

    # Reverse to get the path from start to the meeting state
    path_start.reverse()

    # Path from meeting -> goal
    path_goal = []
    state = meeting_state
    while parent_goal[state][0] is not None:
        parent, move = parent_goal[state]
        # Inverse the move (because it's from goal backwards)
        inverse_move = {"Up": "Down", "Down": "Up", "Left": "Right", "Right": "Left"}[move]
        path_goal.append(inverse_move)
        state = parent

    return path_start + path_goal


## Test the code

In [ ]:
if __name__ == "__main__":

    # Generate the puzzle
    initial_state = generate_random_puzzle()
    print("Initial State:")
    for i in range(0, 9, 3):
        print(initial_state[i:i+3])

    moves, expanded_nodes = iterative_deepening_search(initial_state)

    print ("\nUsing IDS\n")
    print("Sequence of moves:", moves)
    print("Number of moves:", len(moves))
    print("Nodes expanded:", expanded_nodes)

    moves, expanded_nodes = bidirectional_search(initial_state)

    print("\nUsing Bidirectional approach\n")
    print("Sequence of moves:", moves)
    print("Number of moves:", len(moves))
    print("Nodes expanded:", expanded_nodes)


Initial State:
(0, 7, 6)
(2, 5, 8)
(1, 3, 4)

Using IDS

Sequence of moves: ['Down', 'Down', 'Right', 'Right', 'Up', 'Left', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Left', 'Up', 'Right', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Down', 'Right', 'Right']
Number of moves: 24
Nodes expanded: 2742206

Using Bidirectional approach

Sequence of moves: ['Down', 'Right', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Down', 'Right']
Number of moves: 24
Nodes expanded: 1962
